# Camera calibration using a checkerboard.



In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
CHECKERBOARD = (13, 9)

Define 3D World Points (Checkerboard corners)



In [ ]:
objp = []

for y in range(CHECKERBOARD[1]):
    for x in range(CHECKERBOARD[0]):
        objp.append([x, y, 2])

objp = np.array(objp, dtype=np.float32)
# print(objp)

In [ ]:
world_pts = np.array([
    [0, 0, 2],
    [25, 0, 2],
    [50, 0, 2],
    [75, 0, 2],
    [100, 0, 2],
    [125, 0, 2],
    [0, 25, 2],
    [25, 25, 2],
    [50, 25, 2],
    [75, 25, 2],
    [100, 25, 2],
    [125, 25, 2]
], dtype=np.float32)

Load Image and Detect Corners

In [ ]:
image_pts = []

def click_event(event, x, y, flags, param):
    if event == cv2.EVENT_LBUTTONDOWN:
        image_pts.append([x, y])
        print(f"Clicked: {x}, {y}")
        cv2.circle(param, (x,y), 5, (0,0,255), -1)
        cv2.imshow("Select 12 points", param)

img = cv2.imread("img.png")
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
temp = gray.copy()
cv2.imshow("Select 12 points", temp)
cv2.setMouseCallback("Select 12 points", click_event, temp)
cv2.waitKey(0)
cv2.destroyAllWindows()

image_pts = np.array(image_pts, dtype=np.float32)

for pt in image_pts:
    cv2.circle(img, tuple(pt.astype(int)), 12, (0,0,255), -1)
cv2.imwrite("clicked_points.jpg", img)


Clicked: 118, 122
Clicked: 155, 122
Clicked: 191, 146
Clicked: 225, 177
Clicked: 222, 217
Clicked: 225, 254
Clicked: 223, 285
Clicked: 196, 304
Clicked: 166, 354
Clicked: 127, 295
Clicked: 85, 265
Clicked: 121, 171


True

Calibrate the Camera

In [ ]:
world_pts = np.array(world_pts)
image_pts = np.array(image_pts)

A = []
for i in range(len(world_pts)):
    X, Y, Z = world_pts[i]
    u, v = image_pts[i]
    A.append([X, Y, Z, 1, 0, 0, 0, 0, -u*X, -u*Y, -u*Z, -u])
    A.append([0, 0, 0, 0, X, Y, Z, 1, -v*X, -v*Y, -v*Z, -v])

A = np.array(A)

In [ ]:
world_pts

array([[  0.,   0.,   2.],
       [ 25.,   0.,   2.],
       [ 50.,   0.,   2.],
       [ 75.,   0.,   2.],
       [100.,   0.,   2.],
       [125.,   0.,   2.],
       [  0.,  25.,   2.],
       [ 25.,  25.,   2.],
       [ 50.,  25.,   2.],
       [ 75.,  25.,   2.],
       [100.,  25.,   2.],
       [125.,  25.,   2.]], dtype=float32)

In [ ]:
U, S, Vt = np.linalg.svd(A)
P = Vt[-1].reshape(3, 4)

print("Projection Matrix P:\n", P)

Projection Matrix P:
 [[ 5.20487310e-17  1.36786545e-16 -1.66279609e-02  3.32559218e-02]
 [-3.05745013e-17  3.69306365e-16  4.46903218e-01 -8.93806437e-01]
 [-1.24129059e-19  3.75957531e-18 -1.01209681e-03  2.02419362e-03]]


In [ ]:
M = P[:, :3]
K, R = np.linalg.qr(np.linalg.inv(M))
K = np.linalg.inv(K)
R = np.linalg.inv(R)

K = K / K[-1, -1]

t = np.linalg.inv(K) @ P[:, 3]

Save Calibration Parameters

In [ ]:
print("\nIntrinsic Matrix (Mint):\n", K)
print("\nRotation Matrix:\n", R)
print("\nTranslation Vector:\n", t)


Intrinsic Matrix (Mint):
 [[-9.99116051e-01 -4.20370790e-02 -3.36155873e-17]
 [ 4.20370790e-02 -9.99116051e-01 -3.71651211e-15]
 [ 1.22645440e-16 -3.71464001e-15  1.00000000e+00]]

Rotation Matrix:
 [[-5.71938708e-17 -7.26796382e-17 -1.66279609e-02]
 [ 0.00000000e+00 -2.03118640e-15  4.46903218e-01]
 [ 0.00000000e+00  0.00000000e+00 -1.01209681e-03]]

Translation Vector:
 [-0.07079954  0.89161838  0.00202419]
